## CopperHead V2 tutorial

This framework builds upon columnar analysis platform coffea 202x python package, using awkward arrays and dask distributed for parallelization.

First we setup our config by specifying the era/year we will be doing our analysis work on.

# Pre-stage
We run ```make_parameters.py``` while specifying the year with ```--year``` flag. The output of the python script will be saved in ```./config/parameters.json```

In [ ]:
! python make_parameters.py --year 2018

Now we prepare the list of samples that we will be performing our analysis on. This can be done by executing ```run_prestage.py``` script, specifying the chunksize by using ```--chunksize``` flag and listing the samples we would like to perform our analysis on with ```--input_string``` flag.

The chunksize value is simple: it is an integer value of "chunks" of rows of data that each worker works on during parallelized workflow. 

Moreover, one can specify the list of data runs, MC background samples and MC signal samples for the analysis to run on by using --data, --background and --signal flag respectively. If left empty/ imcompatible (ie data 'A' in year 2017), it will just skip and move on.


In [ ]:
data_l = ['A', 'B', 'C', 'D']
bkg_l = ['DY', 'TT',]
sig_l = ['ggH', 'VBF']
! python run_prestage.py --chunksize 80000 --year 2018 --cluster True --data {' '.join(data_l)} --background {' '.join(bkg_l)} --signal {' '.join(sig_l)}

If we wish to run our analysis only onto a subset of our samples in order to save time, for example, we can do so my specifying the fraction of the samples we would like to perform our analysis on with the ```--change_fraction``` flag with the accompanying floating value representing the fraction of the samples we want to work on.

For example running this cell below would trim our  ```./config/fraction_processor_samples.json``` by approximately ten percent.

In [ ]:
! python run_prestage.py --change_fraction 0.1

The code above will only less than a second. This will save a new config file ```./config/fraction_processor_samples.json```. Please note that we don't overwrite the original full config file ```./config/fraction_processor_samples.json```. This is so that if you would like to change your fraction value, you can do so quickly, instead of waiting a full minute to redo the whole prestage step.

# Running Stage 1

Now we're ready to execute stage 1 of the analysis, which refers to the baseline selections we apply just before categorization of Higgs decay categories. we do this by simply running ```run_stage1.py```, though we recommend to also add ```-W ignore``` option to suppress warning flags. This operation takes the most time, ranging from 30 mins for fraction of around 0.25, all the way to hours for a full sample run. The outputs of the ```run_stage1.py``` will be saved as collection of ```.parquet``` files in the directory that's defined in the ```--save_path``` flag along with the sample name and fraction. 

For instance, data_A samples with fraction 0.25 with sample_path of ```/depot/cms/users/yun79/results/stage1/test/``` would be saved at ```/depot/cms/users/yun79/results/stage1/test/f0_25/data_A```

In [ ]:
! python -W ignore run_stage1.py --save_path "/depot/cms/users/yun79/results/stage1/test/"

# Stage 1 Validation
Now we validate our stage 1 outputs by plotting validation histograms. Like ```run_prestage.py``` script, we can specify the options of the plots via ```--input_string``` flag, but with different formating, but this time with mostly just boolean values: 


Ratio_{Y or N}/LogY_{Y or N}/ShowLumi_{Y or N}/Status_{work or prelim}

Where we specify if we want Data/MC ratio plot in the bottom panel on with "Y" to mean yes and "N" to mean no after ```Ratio_```, plot in log scale in the y axis after ```LogY_```, show integrated luminosity value of the run after ```ShowLumi_``` and status of the plot after ```Status_```, where the option is "work" for "Work in Progress", "prelim" for "Preliminary" and empty character ("") for no mention of the status at all.

Ie: Ratio_Y/LogY_Y/ShowLumi_N/Status_work indicates to have Data/MC ratio plot on the bottom, plot in logarithmic scale, don't show the integrated luminosity value, and have "Work in progress" label

next is the ```--load_path``` flag, which should be identical to the path specified in ```--save_path``` flag when running the ```run_stage1.py``` script.

One can also specify the path to where the validation plots will be saved by adding ```--save_path``` flag onto ```run_stage1_validation.py``` script, or just use the default path ```./validation/figs```

In [ ]:
! python run_stage1_validation.py --fraction 0.001 --input_string "Ratio_Y/LogY_Y/ShowLumi_N/Status_work" --load_path "/depot/cms/users/yun79/results/stage1/test/"

In [109]:
# data_l = ['A', 'B', 'C', 'D']
data_l = ['A',  'B', 'C']
bkg_l = ['DY','TT',]
sig_l = ['ggH', 'VBF']
# vars2plot = ['jet', 'mu', 'dimuon'] 
vars2plot = ['jet'] 
lumi = 137.9
status = "Private_Work"

In [121]:
# fraction = 0.1
fraction = 0.001
fraction_str = str(fraction).replace('.', '_')
load_path = f"/depot/cms/users/yun79/results/stage1/test_full3/f{fraction_str}"
# ! python validation_plotter_ROOT.py --load_path {load_path} --data {' '.join(data_l)} --background {' '.join(bkg_l)} --signal {' '.join(sig_l)} --lumi 137.9 
# ! python validation_plotter_unified.py --load_path {load_path} --data {' '.join(data_l)} --background {' '.join(bkg_l)} --signal {' '.join(sig_l)} --lumi 137.9 --status {status} --no_ratio
! python validation_plotter_unified.py --load_path {load_path} --data {' '.join(data_l)} --background {' '.join(bkg_l)} --signal {' '.join(sig_l)} --lumi 137.9 --status {status}  --ROOT_style   

variables2plot: ['jet1_pt', 'jet2_pt', 'jet1_eta', 'jet2_eta', 'jet1_phi', 'jet2_phi']
process: data_A
fraction_weight: 1000.0009602331048
data activated
process: data_B
fraction_weight: 1000.0001417635383
data activated
TROOT::Append:0: RuntimeWarning: Replacing existing TH1: jet1_pt_hist_data (Potential memory leak).
process: data_C
fraction_weight: 1000.0000818285964
data activated
TROOT::Append:0: RuntimeWarning: Replacing existing TH1: jet1_pt_hist_data (Potential memory leak).
process: dy_M-50
fraction_weight: 1000.0041430105877
process: dy_M-100To200
fraction_weight: 1000.0043496895032
DY activated
process: ttjets_dl
fraction_weight: 1000.0
top activated
process: ttjets_sl
fraction_weight: 1000.0
top activated
TROOT::Append:0: RuntimeWarning: Replacing existing TH1: jet1_pt_hist_Top (Potential memory leak).
process: ggh_powheg
fraction_weight: 1000.0
ggH activated
process: vbf_powheg
fraction_weight: 1000.0
VBF activated
ratio activated
Name: num_hist Title: jet1_pt NbinsX: 29
N

In [44]:
! ls /depot/cms/users/yun79/results/stage1/test_full/f0_001/

dy_M-100To200  dy_M-50	ggh_powheg  ttjets_dl  ttjets_sl  vbf_powheg
